<a href="https://colab.research.google.com/github/GonzaloLaChica1/Bias_Model/blob/main/BiasModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Bias/raw_data"

compas_data = ""


for root, dirs, files in os.walk(path):
  for f in files: # Iterate through each individual filename (string)
      adult_test = os.path.join(root,f)
      if "compas-scores-two-years.csv" in f:
        compas_data = os.path.join(root,f)



print(compas_data)


/content/drive/MyDrive/Bias/raw_data/compas-scores-two-years.csv


In [ ]:
df = pd.read_csv(compas_data)
# df.head()
features = ['age', 'priors_count.1', 'v_decile_score']
target = 'two_year_recid'
protected_attr = 'race'

df_filtered = df[df['race'].isin(['African-American', 'Caucasian'])].copy() # take all columns and rows from df dataframe where isin() retrurn true
df_filtered = df_filtered.dropna(subset=features + [target, protected_attr])



# df_filtered = df_filtered.rename(columns={'priors_count.1':'priors_count', 'v_decile_score': 'decile_score' })
df_filtered['race_binary'] = (df_filtered['race'] == 'African-American').astype(int)
print(f"total records: {len(df_filtered)}")
print(f"African-America: {(df_filtered['race']=='African-American').sum()}")
print(f"Caucasina: {(df['race'] == 'Caucasian').sum()}")
df_filtered.head()

df_filtered[['name', 'race', 'priors_count', 'decile_score', 'race_binary', 'two_year_recid']].head(10)

total records: 6150
African-America: 3696
Caucasina: 2454


,name,race,priors_count,decile_score,race_binary,two_year_recid
1,kevon dixon,African-American,0,3,1,1
2,ed philo,African-American,4,4,1,1
3,marcu brown,African-American,1,8,1,0
6,edward riddle,Caucasian,14,6,0,1
8,elizabeth thieme,Caucasian,0,1,0,0
9,bo bradac,Caucasian,1,3,0,1
10,benjamin franc,Caucasian,0,4,0,0
11,ellyaher lanza,African-American,3,6,1,1
12,kortney coleman,Caucasian,0,1,0,0
13,jarrod turbe,African-American,0,4,1,0


In [ ]:
X = df_filtered[features]
y = df_filtered[target]
race = df_filtered['race_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 67.97%


In [ ]:
# calculate fairness matrics



In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# SECTION 2: DATA LOADING

def load_compas_data(filepath=compas_data):
    """Load and prepare COMPAS dataset"""
    print("\n📊 Loading COMPAS Dataset...")

    df = pd.read_csv(filepath)

    print(f"✅ Loaded {len(df):,} records")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    return df

def load_adult_data(train_path=adult_data,
                    test_path=adult_test):
    """Load and prepare UCI Adult dataset"""
    print("\n📊 Loading UCI Adult Dataset...")

    # Column names
    columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
               'marital-status', 'occupation', 'relationship', 'race', 'sex',
               'capital-gain', 'capital-loss', 'hours-per-week',
               'native-country', 'income']

    # Load train and test
    df_train = pd.read_csv(train_path, names=columns,
                          skipinitialspace=True, na_values='?')
    df_test = pd.read_csv(test_path, names=columns,
                         skipinitialspace=True, skiprows=1, na_values='?')

    # Combine
    df = pd.concat([df_train, df_test], ignore_index=True)

    print(f"✅ Loaded {len(df):,} records")
    print(f"   Train: {len(df_train):,} | Test: {len(df_test):,}")

    return df

def load_hmda_data(filepath=hmda_data,
                   sample_size=100000):
    """Load and prepare HMDA dataset"""
    print("\n📊 Loading HMDA Dataset...")

    # Read sample (HMDA is large)
    df = pd.read_csv(filepath, nrows=sample_size)

    print(f"✅ Loaded {len(df):,} records")
    print(f"   Columns: {len(df.columns)}")

    return df


# SECTION 3: EXPLORATORY DATA ANALYSIS


def explore_dataset(df, dataset_name, target_col, protected_attrs):
    """Comprehensive EDA for any dataset"""

    print(f"\n{'='*70}")
    print(f"EXPLORATORY ANALYSIS: {dataset_name}")
    print(f"{'='*70}")

    # Basic info
    print(f"\n📋 Dataset Shape: {df.shape}")
    print(f"\n📋 Data Types:")
    print(df.dtypes.value_counts())

    # Missing values
    print(f"\n⚠️  Missing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing': missing[missing > 0],
        'Percent': missing_pct[missing > 0]
    }).sort_values('Percent', ascending=False)
    if len(missing_df) > 0:
        print(missing_df.head(10))
    else:
        print("✅ No missing values!")

    # Target distribution
    print(f"\n🎯 Target Variable: {target_col}")
    print(df[target_col].value_counts())
    print(f"\nTarget Distribution:")
    print(df[target_col].value_counts(normalize=True).apply(lambda x: f"{x*100:.2f}%"))

    # Protected attributes analysis
    print(f"\n🔒 Protected Attributes Analysis:")
    for attr in protected_attrs:
        if attr in df.columns:
            print(f"\n{attr.upper()}:")
            print(df[attr].value_counts().head(10))
            print(f"Unique values: {df[attr].nunique()}")

    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{dataset_name} - Overview', fontsize=16)

    # Target distribution
    df[target_col].value_counts().plot(kind='bar', ax=axes[0, 0], color='steelblue')
    axes[0, 0].set_title('Target Distribution')
    axes[0, 0].set_xlabel(target_col)
    axes[0, 0].set_ylabel('Count')

    # Protected attribute distributions
    if len(protected_attrs) >= 1 and protected_attrs[0] in df.columns:
        df[protected_attrs[0]].value_counts().head(10).plot(
            kind='bar', ax=axes[0, 1], color='coral')
        axes[0, 1].set_title(f'{protected_attrs[0]} Distribution')
        axes[0, 1].tick_params(axis='x', rotation=45)

    if len(protected_attrs) >= 2 and protected_attrs[1] in df.columns:
        df[protected_attrs[1]].value_counts().plot(
            kind='bar', ax=axes[1, 0], color='lightgreen')
        axes[1, 0].set_title(f'{protected_attrs[1]} Distribution')

    # Missing data heatmap (if any)
    if missing.sum() > 0:
        sns.heatmap(df.isnull(), yticklabels=False, cbar=True,
                   cmap='viridis', ax=axes[1, 1])
        axes[1, 1].set_title('Missing Data Pattern')
    else:
        axes[1, 1].text(0.5, 0.5, 'No Missing Data',
                       ha='center', va='center', fontsize=14)
        axes[1, 1].axis('off')

    plt.tight_layout()
    plt.savefig(f'outputs/{dataset_name}_eda.png', dpi=300, bbox_inches='tight')
    print(f"\n📊 Visualization saved: outputs/{dataset_name}_eda.png")
    plt.savefig(f'/content/drive/MyDrive/Bias/outputs/{dataset_name}_eda.png')
    plt.show()

    return df


# SECTION 4: DATA PREPROCESSING


def preprocess_compas(df):
    """Clean and prepare COMPAS data"""
    print("\n🔧 Preprocessing COMPAS...")

    # Select relevant columns
    cols = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text',
            'sex', 'priors_count', 'days_b_screening_arrest',
            'decile_score', 'is_recid', 'two_year_recid',
            'c_jail_in', 'c_jail_out']

    df = df[cols].copy()

    # Filter data (standard COMPAS filtering)
    df = df[df['days_b_screening_arrest'] <= 30]
    df = df[df['days_b_screening_arrest'] >= -30]
    df = df[df['is_recid'] != -1]
    df = df[df['c_charge_degree'] != "O"]
    df = df[df['score_text'] != 'N/A']

    # Create target
    df['target'] = df['two_year_recid']

    # Encode protected attributes
    df['race_binary'] = df['race'].apply(
        lambda x: 1 if x == 'Caucasian' else 0)
    df['sex_binary'] = df['sex'].apply(
        lambda x: 1 if x == 'Male' else 0)

    # Feature engineering
    df['age_squared'] = df['age'] ** 2
    df['priors_squared'] = df['priors_count'] ** 2

    print(f"✅ Final shape: {df.shape}")
    print(f"   Recidivism rate: {df['target'].mean()*100:.2f}%")

    return df

def preprocess_adult(df):
    """Clean and prepare UCI Adult data"""
    print("\n🔧 Preprocessing UCI Adult...")

    # Remove missing values
    df = df.dropna()

    # Create binary target
    df['target'] = df['income'].apply(lambda x: 1 if '>50K' in x else 0)

    # Encode protected attributes
    df['race_binary'] = df['race'].apply(
        lambda x: 1 if x == 'White' else 0)
    df['sex_binary'] = df['sex'].apply(
        lambda x: 1 if x == 'Male' else 0)

    # Encode categorical variables
    categorical_cols = ['workclass', 'education', 'marital-status',
                       'occupation', 'relationship', 'native-country']

    le_dict = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col])
        le_dict[col] = le

    print(f"✅ Final shape: {df.shape}")
    print(f"   High income rate: {df['target'].mean()*100:.2f}%")

    return df, le_dict

def preprocess_hmda(df):
    """Clean and prepare HMDA data"""
    print("\n🔧 Preprocessing HMDA...")

    # Filter for originated or denied loans
    df = df[df['action_taken'].isin([1, 3])].copy()

    # Create target (1 = approved, 0 = denied)
    df['target'] = df['action_taken'].apply(lambda x: 1 if x == 1 else 0)

    # Encode protected attributes
    if 'derived_race' in df.columns:
        df['race_binary'] = df['derived_race'].apply(
            lambda x: 1 if x == 'White' else 0)

    if 'derived_sex' in df.columns:
        df['sex_binary'] = df['derived_sex'].apply(
            lambda x: 1 if x == 'Male' else 0)

    # Select numeric features
    numeric_features = ['loan_amount', 'income', 'property_value',
                       'debt_to_income_ratio', 'applicant_age']

    # Keep only available features
    numeric_features = [f for f in numeric_features if f in df.columns]

    print(f"✅ Final shape: {df.shape}")
    print(f"   Approval rate: {df['target'].mean()*100:.2f}%")

    return df


# SECTION 5: BASELINE MODEL TRAINING


def prepare_train_test(df, feature_cols, target_col='target',
                       test_size=0.2, random_state=42):
    """Split data into train/test sets"""

    X = df[feature_cols]
    y = df[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y)

    print(f"\n📊 Train/Test Split:")
    print(f"   Training: {len(X_train):,} samples")
    print(f"   Testing: {len(X_test):,} samples")
    print(f"   Positive rate (train): {y_train.mean()*100:.2f}%")
    print(f"   Positive rate (test): {y_test.mean()*100:.2f}%")

    return X_train, X_test, y_train, y_test

def train_baseline_models(X_train, X_test, y_train, y_test, dataset_name):
    """Train Logistic Regression, Random Forest, and XGBoost"""

    print(f"\n{'='*70}")
    print(f"TRAINING BASELINE MODELS: {dataset_name}")
    print(f"{'='*70}")

    results = {}

    # 1. Logistic Regression
    print("\n🔄 Training Logistic Regression...")
    lr = LogisticRegression(max_iter=1000, random_state=42)
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_proba = lr.predict_proba(X_test)[:, 1]

    results['Logistic Regression'] = {
        'model': lr,
        'predictions': lr_pred,
        'probabilities': lr_proba,
        'accuracy': accuracy_score(y_test, lr_pred),
        'precision': precision_score(y_test, lr_pred),
        'recall': recall_score(y_test, lr_pred),
        'f1': f1_score(y_test, lr_pred),
        'auc': roc_auc_score(y_test, lr_proba)
    }
    print(f"✅ Accuracy: {results['Logistic Regression']['accuracy']:.4f}")

    # 2. Random Forest
    print("\n🔄 Training Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_proba = rf.predict_proba(X_test)[:, 1]

    results['Random Forest'] = {
        'model': rf,
        'predictions': rf_pred,
        'probabilities': rf_proba,
        'accuracy': accuracy_score(y_test, rf_pred),
        'precision': precision_score(y_test, rf_pred),
        'recall': recall_score(y_test, rf_pred),
        'f1': f1_score(y_test, rf_pred),
        'auc': roc_auc_score(y_test, rf_proba)
    }
    print(f"✅ Accuracy: {results['Random Forest']['accuracy']:.4f}")

    # 3. XGBoost
    print("\n🔄 Training XGBoost...")
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42,
                                  use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    xgb_proba = xgb_model.predict_proba(X_test)[:, 1]

    results['XGBoost'] = {
        'model': xgb_model,
        'predictions': xgb_pred,
        'probabilities': xgb_proba,
        'accuracy': accuracy_score(y_test, xgb_pred),
        'precision': precision_score(y_test, xgb_pred),
        'recall': recall_score(y_test, xgb_pred),
        'f1': f1_score(y_test, xgb_pred),
        'auc': roc_auc_score(y_test, xgb_proba)
    }
    print(f"✅ Accuracy: {results['XGBoost']['accuracy']:.4f}")

    # Summary comparison
    print(f"\n{'='*70}")
    print("MODEL COMPARISON")
    print(f"{'='*70}")

    comparison_df = pd.DataFrame({
        'Model': results.keys(),
        'Accuracy': [r['accuracy'] for r in results.values()],
        'Precision': [r['precision'] for r in results.values()],
        'Recall': [r['recall'] for r in results.values()],
        'F1-Score': [r['f1'] for r in results.values()],
        'AUC-ROC': [r['auc'] for r in results.values()]
    })

    print(comparison_df.to_string(index=False))

    # Visualize comparison
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Metrics comparison
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
    x = np.arange(len(metrics))
    width = 0.25

    for i, (model_name, result) in enumerate(results.items()):
        values = [result['accuracy'], result['precision'],
                 result['recall'], result['f1'], result['auc']]
        axes[0].bar(x + i*width, values, width, label=model_name)

    axes[0].set_xlabel('Metrics')
    axes[0].set_ylabel('Score')
    axes[0].set_title(f'{dataset_name} - Model Performance Comparison')
    axes[0].set_xticks(x + width)
    axes[0].set_xticklabels(metrics)
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)

    # Confusion matrices
    for i, (model_name, result) in enumerate(results.items()):
        cm = confusion_matrix(y_test, result['predictions'])

        if i == 0:
            ax = axes[1]
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
            ax.set_title(f'{model_name}\nConfusion Matrix')
            ax.set_ylabel('True Label')
            ax.set_xlabel('Predicted Label')

    plt.tight_layout()
    plt.savefig(f'outputs/{dataset_name}_baseline_models.png',
                dpi=300, bbox_inches='tight')
    print(f"\n📊 Visualization saved: outputs/{dataset_name}_baseline_models.png")
    plt.savefig(f'/content/drive/MyDrive/Bias/outputs/{dataset_name}_baseline_models.png')
    plt.show()


    return results, comparison_df


# SECTION 6: INITIAL BIAS CHECK


def check_initial_bias(df, predictions, protected_attr, target_col='target'):
    """Quick bias check across protected attribute"""

    print(f"\n{'='*70}")
    print(f"INITIAL BIAS CHECK: {protected_attr}")
    print(f"{'='*70}")

    df_check = df.copy()
    df_check['prediction'] = predictions

    # Group by protected attribute
    groups = df_check[protected_attr].unique()

    print(f"\n📊 Outcome Rates by {protected_attr}:")
    for group in groups:
        group_data = df_check[df_check[protected_attr] == group]

        actual_rate = group_data[target_col].mean()
        pred_rate = group_data['prediction'].mean()

        print(f"\n  Group {group}:")
        print(f"    Sample size: {len(group_data):,}")
        print(f"    Actual positive rate: {actual_rate*100:.2f}%")
        print(f"    Predicted positive rate: {pred_rate*100:.2f}%")

    # Calculate disparate impact
    if len(groups) == 2:
        group0 = df_check[df_check[protected_attr] == groups[0]]
        group1 = df_check[df_check[protected_attr] == groups[1]]

        rate0 = group0['prediction'].mean()
        rate1 = group1['prediction'].mean()

        di_ratio = min(rate0, rate1) / max(rate0, rate1)

        print(f"\n⚖️  Disparate Impact Ratio: {di_ratio:.3f}")
        if di_ratio < 0.8:
            print("   ⚠️  WARNING: Below 0.80 threshold (potential discrimination)")
        else:
            print("   ✅ Above 0.80 threshold")

    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Prediction rates by group
    pred_rates = df_check.groupby(protected_attr)['prediction'].mean()
    pred_rates.plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title(f'Positive Prediction Rate by {protected_attr}')
    axes[0].set_ylabel('Rate')
    axes[0].axhline(y=0.8*pred_rates.max(), color='r',
                    linestyle='--', label='80% threshold')
    axes[0].legend()

    # Actual vs predicted by group
    comparison = df_check.groupby(protected_attr).agg({
        target_col: 'mean',
        'prediction': 'mean'
    })
    comparison.plot(kind='bar', ax=axes[1])
    axes[1].set_title(f'Actual vs Predicted Rates by {protected_attr}')
    axes[1].set_ylabel('Rate')
    axes[1].legend(['Actual', 'Predicted'])

    plt.tight_layout()
    plt.show()
    plt.savefig(f'/content/drive/MyDrive/Bias/outputs/{protected_attr}_bias_check.png',
                dpi=300, bbox_inches='tight')


# MAIN EXECUTION


if __name__ == "__main__":

    print("\n" + "="*70)
    print("FEDERAL AI BIAS RESEARCH - WEEK 1")
    print("Data Exploration & Baseline Models")
    print("="*70)

    # Create output directory
    import os
    os.makedirs('outputs', exist_ok=True)


    # COMPAS DATASET


    print("\n\n" + "#"*70)
    print("# DATASET 1: COMPAS (Criminal Justice)")
    print("#"*70)

    # Load
    compas_df = load_compas_data(compas_data)

    # Explore
    compas_df = explore_dataset(
        compas_df,
        'COMPAS',
        target_col='two_year_recid',
        protected_attrs=['race', 'sex', 'age_cat']
    )

    # Preprocess
    compas_clean = preprocess_compas(compas_df)

    # Define features
    compas_features = ['age', 'priors_count', 'age_squared',
                      'priors_squared', 'c_charge_degree']

    # Encode categorical
    compas_clean['c_charge_degree'] = LabelEncoder().fit_transform(
        compas_clean['c_charge_degree'])

    # Train/test split
    X_train_c, X_test_c, y_train_c, y_test_c = prepare_train_test(
        compas_clean, compas_features)

    # Train models
    compas_results, compas_comparison = train_baseline_models(
        X_train_c, X_test_c, y_train_c, y_test_c, 'COMPAS')

    # Check bias
    check_initial_bias(
        compas_clean.loc[X_test_c.index], # Fix: Changed .iloc to .loc
        compas_results['XGBoost']['predictions'],
        'race_binary',
        'target'
    )


    # UCI ADULT DATASET


    print("\n\n" + "#"*70)
    print("# DATASET 2: UCI ADULT (Benefits/Economic)")
    print("#"*70)

    # Load
    adult_df = load_adult_data(
        adult_data,
        adult_test
    )

    # Explore
    adult_df = explore_dataset(
        adult_df,
        'UCI_Adult',
        target_col='income',
        protected_attrs=['race', 'sex', 'age']
    )

    # Preprocess
    adult_clean, adult_encoders = preprocess_adult(adult_df)

    # Define features
    adult_features = ['age', 'education-num', 'hours-per-week',
                     'capital-gain', 'capital-loss',
                     'workclass_encoded', 'marital-status_encoded',
                     'occupation_encoded', 'relationship_encoded']

    # Train/test split
    X_train_a, X_test_a, y_train_a, y_test_a = prepare_train_test(
        adult_clean, adult_features)

    # Train models
    adult_results, adult_comparison = train_baseline_models(
        X_train_a, X_test_a, y_train_a, y_test_a, 'UCI_Adult')

    # Check bias
    check_initial_bias(
        adult_clean.loc[X_test_a.index], # Fix: Changed .iloc to .loc
        adult_results['XGBoost']['predictions'],
        'race_binary',
        'target'
    )


    # HMDA DATASET


    print("\n\n" + "#"*70)
    print("# DATASET 3: HMDA (Lending/Mortgage)")
    print("#"*70)

    try:
        # Load
        hmda_df = load_hmda_data(hmda_data,
                                 sample_size=100000)

        # Explore
        hmda_df = explore_dataset(
            hmda_df,
            'HMDA',
            target_col='action_taken',
            protected_attrs=['derived_race', 'derived_sex',
                           'derived_ethnicity', 'applicant_age']
        )

        # Preprocess
        hmda_clean = preprocess_hmda(hmda_df)

        # Define features (adjust based on available columns)
        hmda_features = []

        # Add features that exist in the dataset
        possible_features = ['loan_amount', 'income', 'property_value',
                           'debt_to_income_ratio', 'loan_term',
                           'interest_rate', 'rate_spread',
                           'loan_to_value_ratio', 'combined_loan_to_value_ratio']

        for feat in possible_features:
            if feat in hmda_clean.columns:
                # Handle missing values
                hmda_clean[feat] = hmda_clean[feat].fillna(
                    hmda_clean[feat].median())
                hmda_features.append(feat)

        # Encode categorical features if needed
        if 'loan_type' in hmda_clean.columns:
            hmda_clean['loan_type_encoded'] = LabelEncoder().fit_transform(
                hmda_clean['loan_type'].fillna('Unknown'))
            hmda_features.append('loan_type_encoded')

        if 'loan_purpose' in hmda_clean.columns:
            hmda_clean['loan_purpose_encoded'] = LabelEncoder().fit_transform(
                hmda_clean['loan_purpose'].fillna('Unknown'))
            hmda_features.append('loan_purpose_encoded')

        print(f"\n📋 Using {len(hmda_features)} features: {hmda_features}")

        # Drop rows with missing target or features
        hmda_clean = hmda_clean.dropna(subset=['target'] + hmda_features)

        # Train/test split
        X_train_h, X_test_h, y_train_h, y_test_h = prepare_train_test(
            hmda_clean, hmda_features)

        # Train models
        hmda_results, hmda_comparison = train_baseline_models(
            X_train_h, X_test_h, y_train_h, y_test_h, 'HMDA')

        # Check bias - Race
        if 'race_binary' in hmda_clean.columns:
            print("\n" + "="*70)
            print("BIAS CHECK: RACE")
            print("="*70)
            check_initial_bias(
                hmda_clean.loc[X_test_h.index], # Fix: Changed .iloc to .loc
                hmda_results['XGBoost']['predictions'],
                'race_binary',
                'target'
            )

        # Check bias - Gender
        if 'sex_binary' in hmda_clean.columns:
            print("\n" + "="*70)
            print("BIAS CHECK: GENDER")
            print("="*70)
            check_initial_bias(
                hmda_clean.loc[X_test_h.index], # Fix: Changed .iloc to .loc
                hmda_results['XGBoost']['predictions'],
                'sex_binary',
                'target'
            )

        print("\n✅ HMDA analysis complete!")
        hmda_completed = True

    except FileNotFoundError:
        print("\n⚠️  HMDA file not found. Skipping HMDA analysis.")
        print("   This is optional - you can continue with COMPAS and Adult datasets.")
        hmda_completed = False
    except Exception as e:
        print(f"\n⚠️  Error processing HMDA: {e}")
        print("   Continuing with COMPAS and Adult results.")
        hmda_completed = False


    # SUMMARY


    print("\n\n" + "="*70)
    print("WEEK 1 COMPLETE - SUMMARY")
    print("="*70)

    datasets_completed = 2 + (1 if hmda_completed else 0)

    print(f"\n✅ Completed Tasks:")
    print(f"   1. Loaded {datasets_completed} datasets successfully")
    print(f"   2. Performed exploratory data analysis")
    print(f"   3. Preprocessed and cleaned data")
    print(f"   4. Trained 3 baseline models per dataset")
    print(f"   5. Evaluated model performance")
    print(f"   6. Conducted initial bias assessment")

    print("\n📊 Datasets Analyzed:")
    print("   ✅ COMPAS (Criminal Justice)")
    print("   ✅ UCI Adult (Benefits/Economic)")
    if hmda_completed:
        print("   ✅ HMDA (Lending/Mortgage)")
    else:
        print("   ⚠️  HMDA (Skipped - optional)")

    print("\n📊 Outputs Generated:")
    print("   - EDA visualizations")
    print("   - Model performance comparisons")
    print("   - Initial bias reports")
    print("   - Check the 'outputs/' folder for all visualizations")

    print("\n📝 Next Steps (Week 2):")
    print("   1. Install AIF360 fairness library")
    print("   2. Calculate comprehensive fairness metrics")
    print("   3. Document baseline bias measurements")
    print("   4. Prepare for mitigation techniques")

    print("\n" + "="*70)
    print("Save your work and review the outputs folder!")
    print("="*70)



# SAVing RESULTS FOR WEEK 2


print("\n💾 Saving results for Week 2...")

# Save cleaned datasets
compas_clean.to_csv('/content/drive/MyDrive/Bias/outputs/compas_clean.csv', index=False)
adult_clean.to_csv('/content/drive/MyDrive/Bias/outputs/adult_clean.csv', index=False)

# Save predictions
pd.DataFrame({
    'prediction': compas_results['XGBoost']['predictions'],
    'test_index': X_test_c.index
}).to_csv('/content/drive/MyDrive/Bias/outputs/compas_predictions.csv', index=False)

pd.DataFrame({
    'prediction': adult_results['XGBoost']['predictions'],
    'test_index': X_test_a.index
}).to_csv('/content/drive/MyDrive/Bias/outputs/adult_predictions.csv', index=False)

# If you have HMDA
if hmda_completed:
    hmda_clean.to_csv('/content/drive/MyDrive/Bias/outputs/hmda_clean.csv', index=False)
    pd.DataFrame({
        'prediction': hmda_results['XGBoost']['predictions'],
        'test_index': X_test_h.index
    }).to_csv('/content/drive/MyDrive/Bias/outputs/hmda_predictions.csv', index=False)

print("✅ All results saved to outputs/ folder")

NameError: name 'adult_data' is not defined

In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=100,use_label_encoder=False, random_state=42, eval_metric='logloss')

my_adult_df = pd.read_csv("/content/drive/MyDrive/Bias/processed_data/adult.csv")
my_adult_df.head()
adult_features = ['age', 'education-num', 'hours-per-week',
                     'capital-gain', 'capital-loss',
                     'workclass_encoded', 'marital-status_encoded',
                     'occupation_encoded', 'relationship_encoded']
X = my_adult_df[adult_features]
y = my_adult_df['income']

# Clean the 'income' column by removing periods and stripping whitespace
y = y.astype(str).str.replace('.', '', regex=False).str.strip()



# Now the mapping
y = y.map({'<=50K': 0, '>50K': 1, "0": 0, "1": 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))


Unique values in 'y' after cleaning but before mapping: <class 'str'>
              precision    recall  f1-score   support

           0       0.89      0.94      0.91      7170
           1       0.79      0.64      0.70      2355

    accuracy                           0.87      9525
   macro avg       0.84      0.79      0.81      9525
weighted avg       0.86      0.87      0.86      9525



In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Bias/outputs"
compas_clean = ""
compas_prediction = ""
adult_clean = ""
adult_prediction = ""
for root, file, filesname in os.walk(path):
  for f in filesname:
    if "adult_clean" in f:
      adult_clean = os.path.join(root, f)
    if "adult_prediction" in f:
      adult_prediction = os.path.join(root, f)
    if "compas_clean" in f:
      compas_clean = os.path.join(root, f)
    if "compas_prediction" in f:
      compas_prediction = os.path.join(root, f)
try:
  print(adult_clean)
  print(adult_prediction)
  print(compas_clean)
  print(compas_prediction)
except NameError:
  print("error")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Bias/outputs/adult_clean.csv
/content/drive/MyDrive/Bias/outputs/adult_predictions.csv
/content/drive/MyDrive/Bias/outputs/compas_clean.csv
/content/drive/MyDrive/Bias/outputs/compas_predictions.csv


In [ ]:
#2nd

!pip install aif360
!pip install fairlearn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

try:
    from aif360.datasets import BinaryLabelDataset
    from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
    from aif360.explainers import MetricTextExplainer
    print("✅ AIF360 imported successfully!")
except ImportError:
    print("❌ AIF360 not found. Install with: pip install aif360")
    exit()

# Import Fairlearn for additional metrics
try:
    from fairlearn.metrics import (
        demographic_parity_difference,
        demographic_parity_ratio,
        equalized_odds_difference
    )
    print("✅ Fairlearn imported successfully!")
except ImportError:
    print("⚠️  Fairlearn not found. Install with: pip install fairlearn")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)


# SECTION 2: HELPER FUNCTIONS FOR AIF360


def create_aif360_dataset(df, label_name, favorable_label,
                          protected_attribute_names,
                          privileged_protected_attributes):
    """
    Convert pandas DataFrame to AIF360 BinaryLabelDataset

    Parameters:
    -----------
    df : pandas DataFrame
    label_name : str - name of target column
    favorable_label : int - positive class (1 for approved, 0 for denied)
    protected_attribute_names : list - ['race', 'sex']
    privileged_protected_attributes : list of dicts - [{'race': 1}, {'sex': 1}]
    """

    return BinaryLabelDataset(
        favorable_label=favorable_label,
        unfavorable_label=1 - favorable_label,
        df=df,
        label_names=[label_name],
        protected_attribute_names=protected_attribute_names,
        privileged_protected_attributes=privileged_protected_attributes
    )

def calculate_all_fairness_metrics(dataset_true, dataset_pred,
                                   unprivileged_groups, privileged_groups,
                                   dataset_name="Dataset"):
    """
    Calculate comprehensive fairness metrics using AIF360

    Returns: Dictionary with all metrics
    """

    print(f"\n{'='*70}")
    print(f"FAIRNESS METRICS ANALYSIS: {dataset_name}")
    print(f"{'='*70}")

    # Create metrics object
    metric = ClassificationMetric(
        dataset_true,
        dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )

    metrics_dict = {}


    # GROUP FAIRNESS METRICS


    print("\n📊 GROUP FAIRNESS METRICS")
    print("-" * 70)

    # 1. Statistical Parity Difference (Demographic Parity)
    spd = metric.statistical_parity_difference()
    metrics_dict['statistical_parity_difference'] = spd
    print(f"\n1. Statistical Parity Difference: {spd:.4f}")
    print(f"   Interpretation: Difference in positive prediction rates")
    print(f"   Ideal value: 0 (no difference)")
    print(f"   Threshold: |SPD| < 0.10 is acceptable")
    if abs(spd) > 0.10:
        print(f"   ⚠️  WARNING: Exceeds fairness threshold!")
    else:
        print(f"   ✅ Within acceptable range")

    # 2. Disparate Impact
    di = metric.disparate_impact()
    metrics_dict['disparate_impact'] = di
    print(f"\n2. Disparate Impact Ratio: {di:.4f}")
    print(f"   Interpretation: Ratio of positive rates (unprivileged/privileged)")
    print(f"   Ideal value: 1.0 (equal rates)")
    print(f"   Legal threshold: DI ≥ 0.80 (EEOC 80% rule)")
    if di < 0.80:
        print(f"   ⚠️  LEGAL VIOLATION: Below 0.80 threshold!")
    elif di > 1.25:
        print(f"   ⚠️  WARNING: Reverse discrimination (DI > 1.25)")
    else:
        print(f"   ✅ Within legal bounds")

    # 3. Equal Opportunity Difference (TPR difference)
    eod = metric.equal_opportunity_difference()
    metrics_dict['equal_opportunity_difference'] = eod
    print(f"\n3. Equal Opportunity Difference: {eod:.4f}")
    print(f"   Interpretation: Difference in True Positive Rates")
    print(f"   Ideal value: 0 (equal TPR across groups)")
    print(f"   Measures: Are qualified people from both groups accepted equally?")
    if abs(eod) > 0.10:
        print(f"   ⚠️  Significant difference in opportunity")
    else:
        print(f"   ✅ Similar opportunity across groups")

    # 4. Average Odds Difference (Equalized Odds)
    aod = metric.average_odds_difference()
    metrics_dict['average_odds_difference'] = aod
    print(f"\n4. Average Odds Difference: {aod:.4f}")
    print(f"   Interpretation: Average of TPR and FPR differences")
    print(f"   Ideal value: 0 (equal error rates)")
    print(f"   Measures: Overall fairness in predictions")
    if abs(aod) > 0.10:
        print(f"   ⚠️  Significant difference in error rates")
    else:
        print(f"   ✅ Similar error rates across groups")

    # 5. Theil Index (measures inequality)
    theil = metric.theil_index()
    metrics_dict['theil_index'] = theil
    print(f"\n5. Theil Index: {theil:.4f}")
    print(f"   Interpretation: Measures inequality in benefit allocation")
    print(f"   Ideal value: 0 (perfect equality)")
    print(f"   Range: [0, ∞)")


    # INDIVIDUAL FAIRNESS METRICS


    print("\n\n📊 INDIVIDUAL FAIRNESS METRICS")
    print("-" * 70)

    # 6. Consistency Score
    # Note: Requires individual fairness calculation (computationally expensive)
    # We'll calculate a proxy using prediction variance within similar groups

    print("\n6. Consistency Analysis")
    print("   (Simplified proxy - full individual fairness requires k-NN)")
    print("   Measures: Similar individuals get similar predictions")


    # PERFORMANCE METRICS BY GROUP


    print("\n\n📊 PERFORMANCE METRICS BY GROUP")
    print("-" * 70)

    # True Positive Rates
    tpr_priv = metric.true_positive_rate(privileged=True)
    tpr_unpriv = metric.true_positive_rate(privileged=False)
    metrics_dict['tpr_privileged'] = tpr_priv
    metrics_dict['tpr_unprivileged'] = tpr_unpriv

    print(f"\n7. True Positive Rate (Sensitivity/Recall)")
    print(f"   Privileged group: {tpr_priv:.4f}")
    print(f"   Unprivileged group: {tpr_unpriv:.4f}")
    print(f"   Difference: {abs(tpr_priv - tpr_unpriv):.4f}")

    # False Positive Rates
    fpr_priv = metric.false_positive_rate(privileged=True)
    fpr_unpriv = metric.false_positive_rate(privileged=False)
    metrics_dict['fpr_privileged'] = fpr_priv
    metrics_dict['fpr_unprivileged'] = fpr_unpriv

    print(f"\n8. False Positive Rate")
    print(f"   Privileged group: {fpr_priv:.4f}")
    print(f"   Unprivileged group: {fpr_unpriv:.4f}")
    print(f"   Difference: {abs(fpr_priv - fpr_unpriv):.4f}")

    # Positive Prediction Rates
    ppr_priv = metric.positive_rate(privileged=True)
    ppr_unpriv = metric.positive_rate(privileged=False)
    metrics_dict['positive_rate_privileged'] = ppr_priv
    metrics_dict['positive_rate_unprivileged'] = ppr_unpriv

    print(f"\n9. Positive Prediction Rate")
    print(f"   Privileged group: {ppr_priv:.4f}")
    print(f"   Unprivileged group: {ppr_unpriv:.4f}")
    print(f"   Difference: {abs(ppr_priv - ppr_unpriv):.4f}")

    # Selection Rates
    print(f"\n10. Selection Rate Analysis")
    print(f"    % Selected (Privileged): {ppr_priv*100:.2f}%")
    print(f"    % Selected (Unprivileged): {ppr_unpriv*100:.2f}%")
    print(f"    Ratio: {ppr_unpriv/ppr_priv if ppr_priv > 0 else 0:.4f}")

    # Accuracy by group
    acc_priv = metric.accuracy(privileged=True)
    acc_unpriv = metric.accuracy(privileged=False)
    metrics_dict['accuracy_privileged'] = acc_priv
    metrics_dict['accuracy_unprivileged'] = acc_unpriv

    print(f"\n11. Accuracy by Group")
    print(f"    Privileged group: {acc_priv:.4f}")
    print(f"    Unprivileged group: {acc_unpriv:.4f}")
    print(f"    Difference: {abs(acc_priv - acc_unpriv):.4f}")

    # ========================================================================
    # BIAS SEVERITY ASSESSMENT
    # ========================================================================

    print("\n\n" + "="*70)
    print("BIAS SEVERITY ASSESSMENT")
    print("="*70)

    severity_score = 0
    issues = []

    if abs(spd) > 0.10:
        severity_score += 2
        issues.append("Statistical parity violated")

    if di < 0.80 or di > 1.25:
        severity_score += 3  # Legal threshold
        issues.append("Disparate impact violation (LEGAL ISSUE)")

    if abs(eod) > 0.10:
        severity_score += 2
        issues.append("Unequal opportunity detected")

    if abs(aod) > 0.10:
        severity_score += 2
        issues.append("Unequal error rates")

    if abs(tpr_priv - tpr_unpriv) > 0.15:
        severity_score += 1
        issues.append("Large TPR difference")

    if abs(fpr_priv - fpr_unpriv) > 0.15:
        severity_score += 1
        issues.append("Large FPR difference")

    print(f"\nBias Severity Score: {severity_score}/11")
    print(f"Severity Level: ", end="")

    if severity_score == 0:
        print("✅ MINIMAL - No significant bias detected")
    elif severity_score <= 3:
        print("⚠️  LOW - Minor fairness concerns")
    elif severity_score <= 6:
        print("⚠️  MODERATE - Significant bias, mitigation recommended")
    elif severity_score <= 9:
        print("🚨 HIGH - Serious bias, mitigation required")
    else:
        print("🚨 CRITICAL - Severe bias, immediate action required")

    if issues:
        print(f"\nIssues detected:")
        for i, issue in enumerate(issues, 1):
            print(f"  {i}. {issue}")

    return metrics_dict

# ============================================================================
# SECTION 3: VISUALIZATION FUNCTIONS


def visualize_fairness_metrics(metrics_dict, dataset_name):
    """Create comprehensive fairness visualization"""

    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'{dataset_name} - Comprehensive Fairness Analysis',
                 fontsize=16, fontweight='bold')

    # 1. Disparate Impact
    ax = axes[0, 0]
    di = metrics_dict['disparate_impact']
    colors = ['red' if di < 0.8 else 'yellow' if di < 0.9 else 'green']
    ax.bar(['Disparate\nImpact'], [di], color=colors[0], alpha=0.7)
    ax.axhline(y=0.8, color='red', linestyle='--', label='Legal threshold')
    ax.axhline(y=1.0, color='green', linestyle='--', label='Perfect fairness')
    ax.set_ylabel('Ratio')
    ax.set_title('Disparate Impact\n(Legal Threshold: 0.80)')
    ax.legend()
    ax.set_ylim([0, max(1.5, di + 0.2)])

    # 2. Statistical Parity
    ax = axes[0, 1]
    spd = metrics_dict['statistical_parity_difference']
    color = 'red' if abs(spd) > 0.1 else 'green'
    ax.bar(['Statistical Parity\nDifference'], [spd], color=color, alpha=0.7)
    ax.axhline(y=0, color='green', linestyle='--', label='Perfect parity')
    ax.axhline(y=0.1, color='orange', linestyle='--', label='Threshold')
    ax.axhline(y=-0.1, color='orange', linestyle='--')
    ax.set_ylabel('Difference')
    ax.set_title('Statistical Parity Difference\n(Threshold: \u00b10.10)')
    ax.legend()

    # 3. True Positive Rates
    ax = axes[0, 2]
    tpr_data = [metrics_dict['tpr_privileged'],
                metrics_dict['tpr_unprivileged']]
    bars = ax.bar(['Privileged', 'Unprivileged'], tpr_data,
                   color=['steelblue', 'coral'], alpha=0.7)
    ax.set_ylabel('Rate')
    ax.set_title('True Positive Rate by Group')
    ax.set_ylim([0, 1])
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    # 4. False Positive Rates
    ax = axes[1, 0]
    fpr_data = [metrics_dict['fpr_privileged'],
                metrics_dict['fpr_unprivileged']]
    bars = ax.bar(['Privileged', 'Unprivileged'], fpr_data,
                   color=['steelblue', 'coral'], alpha=0.7)
    ax.set_ylabel('Rate')
    ax.set_title('False Positive Rate by Group')
    ax.set_ylim([0, 1])
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    # 5. Selection Rates
    ax = axes[1, 1]
    sel_data = [metrics_dict['positive_rate_privileged'],
                metrics_dict['positive_rate_unprivileged']]
    bars = ax.bar(['Privileged', 'Unprivileged'], sel_data,
                   color=['steelblue', 'coral'], alpha=0.7)
    ax.set_ylabel('Rate')
    ax.set_title('Positive Prediction Rate\n(Selection Rate)')
    ax.set_ylim([0, 1])
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    # 6. Accuracy by Group
    ax = axes[1, 2]
    acc_data = [metrics_dict['accuracy_privileged'],
                metrics_dict['accuracy_unprivileged']]
    bars = ax.bar(['Privileged', 'Unprivileged'], acc_data,
                   color=['steelblue', 'coral'], alpha=0.7)
    ax.set_ylabel('Accuracy')
    ax.set_title('Model Accuracy by Group')
    ax.set_ylim([0, 1])
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig(f'outputs/{dataset_name}_fairness_metrics.png',
                dpi=300, bbox_inches='tight')
    print(f"\n📊 Visualization saved: outputs/{dataset_name}_fairness_metrics.png")
    plt.show()

def create_fairness_report(all_metrics, dataset_names):
    """Create summary table of all fairness metrics"""

    print("\n" + "="*70)
    print("FAIRNESS METRICS COMPARISON ACROSS DATASETS")
    print("="*70)

    metrics_to_compare = [
        'disparate_impact',
        'statistical_parity_difference',
        'equal_opportunity_difference',
        'average_odds_difference',
        'tpr_unprivileged',
        'fpr_unprivileged'
    ]

    comparison_data = []
    for dataset_name in dataset_names:
        row = {'Dataset': dataset_name}
        for metric in metrics_to_compare:
            if metric in all_metrics[dataset_name]:
                row[metric] = all_metrics[dataset_name][metric]
        comparison_data.append(row)

    df_comparison = pd.DataFrame(comparison_data)

    # Rename columns for readability
    df_comparison.columns = [
        'Dataset', 'Disparate Impact', 'Stat. Parity Diff',
        'Equal Opp. Diff', 'Avg Odds Diff',
        'TPR (Unpriv)', 'FPR (Unpriv)'
    ]

    print("\n" + df_comparison.to_string(index=False))

    # Save to CSV
    df_comparison.to_csv('outputs/fairness_metrics_comparison.csv', index=False)
    print("\n💾 Saved: outputs/fairness_metrics_comparison.csv")

    return df_comparison


# SECTION 4: MAIN ANALYSIS PIPELINE


def analyze_compas_fairness(df, predictions, model_name="XGBoost"):
    """Comprehensive fairness analysis for COMPAS dataset"""

    print("\n" + "#"*70)
    print("# COMPAS FAIRNESS ANALYSIS")
    print("#"*70)

    # Prepare data
    df_analysis = df.copy()
    df_analysis['prediction'] = predictions

    # The dataframe for AIF360 should only contain the actual features,
    # the protected attributes, and *one* of the label columns (target or prediction).
    # This ensures that 'target' isn't treated as a feature when 'prediction' is the label, and vice versa.

    # Determine common features (excluding 'target' and 'prediction')
    common_features = [col for col in df_analysis.columns if col not in ['target', 'prediction']]
    protected_attribute_names_compas = ['race_binary'] # From the original code context for COMPAS

    # DataFrame for true labels (dataset_true)
    # It should include common features, protected attributes, and the 'target' column
    df_true_for_aif = df_analysis[common_features + protected_attribute_names_compas + ['target']]

    # DataFrame for predicted labels (dataset_pred)
    # It should include common features, protected attributes, and the 'prediction' column
    df_pred_for_aif = df_analysis[common_features + protected_attribute_names_compas + ['prediction']]

    # Create AIF360 datasets
    dataset_true = create_aif360_dataset(
        df_true_for_aif,
        label_name='target',
        favorable_label=0,  # 0 = no recidivism (good outcome)
        protected_attribute_names=protected_attribute_names_compas,
        privileged_protected_attributes=[{'race_binary': 1}]  # 1 = White
    )

    dataset_pred = create_aif360_dataset(
        df_pred_for_aif,
        label_name='prediction',
        favorable_label=0,
        protected_attribute_names=protected_attribute_names_compas,
        privileged_protected_attributes=[{'race_binary': 1}]
    )

    # Calculate metrics
    metrics = calculate_all_fairness_metrics(
        dataset_true,
        dataset_pred,
        unprivileged_groups=[{'race_binary': 0}],  # Non-white
        privileged_groups=[{'race_binary': 1}],    # White
        dataset_name=f"COMPAS ({model_name})"
    )

    # Visualize
    visualize_fairness_metrics(metrics, f"COMPAS_{model_name}")

    return metrics

def analyze_adult_fairness(df, predictions, model_name="XGBoost"):
    """Comprehensive fairness analysis for UCI Adult dataset"""

    print("\n" + "#"*70)
    print("# UCI ADULT FAIRNESS ANALYSIS")
    print("#"*70)

    # Prepare data
    df_analysis = df.copy()
    df_analysis['prediction'] = predictions

    # Determine common features (excluding 'target' and 'prediction')
    common_features = [col for col in df_analysis.columns if col not in ['target', 'prediction']]
    protected_attribute_names_adult = ['race_binary'] # Using race_binary as in initial bias check

    # DataFrame for true labels (dataset_true)
    df_true_for_aif = df_analysis[common_features + protected_attribute_names_adult + ['target']]

    # DataFrame for predicted labels (dataset_pred)
    df_pred_for_aif = df_analysis[common_features + protected_attribute_names_adult + ['prediction']]

    # Create AIF360 datasets
    dataset_true = create_aif360_dataset(
        df_true_for_aif,
        label_name='target',
        favorable_label=1,  # 1 = high income (good outcome)
        protected_attribute_names=protected_attribute_names_adult,
        privileged_protected_attributes=[{'race_binary': 1}]  # 1 = White
    )

    dataset_pred = create_aif360_dataset(
        df_pred_for_aif,
        label_name='prediction',
        favorable_label=1,
        protected_attribute_names=protected_attribute_names_adult,
        privileged_protected_attributes=[{'race_binary': 1}]
    )

    # Calculate metrics
    metrics = calculate_all_fairness_metrics(
        dataset_true,
        dataset_pred,
        unprivileged_groups=[{'race_binary': 0}],  # Non-white
        privileged_groups=[{'race_binary': 1}],    # White
        dataset_name=f"UCI_Adult ({model_name})"
    )

    # Visualize
    visualize_fairness_metrics(metrics, f"UCI_Adult_{model_name}")

    return metrics


# MAIN EXECUTION


if __name__ == "__main__":

    print("\n" + "="*70)
    print("FEDERAL AI BIAS RESEARCH - WEEK 2")
    print("Comprehensive Bias Detection & Fairness Metrics")
    print("="*70)

    print("\n⚠️  NOTE: This script requires Week 1 data.")
    print("Make sure you have:")
    print("  1. Trained models from Week 1")
    print("  2. Test predictions saved")
    print("  3. Preprocessed datasets")


    # LOAD WEEK 1 RESULTS


    print("\n📂 Loading Week 1 results...")

    # You'll need to save these from Week 1 or re-run Week 1 preprocessing
    # For now, this is a template showing what you need

    """
    Example of what you need from Week 1:

    - compas_clean: preprocessed COMPAS dataframe
    - compas_predictions: XGBoost predictions on test set
    - compas_test_indices: indices of test samples

    - adult_clean: preprocessed Adult dataframe
    - adult_predictions: XGBoost predictions on test set
    - adult_test_indices: indices of test samples
    """

    print("\n" + "="*70)
    print("INSTRUCTIONS FOR RUNNING WEEK 2")
    print("="*70)

    print("""
    To run this analysis, you need to:

    1. Run Week 1 code first and save these variables:
       - Preprocessed dataframes
       - Model predictions
       - Test set indices

    2. Then load them here and run fairness analysis

    Example code to add at end of Week 1:

    # Save for Week 2
    compas_clean.to_csv('outputs/compas_clean.csv', index=False)
    adult_clean.to_csv('outputs/adult_clean.csv', index=False)

    pd.DataFrame({
        'prediction': compas_results['XGBoost']['predictions']
    }).to_csv('outputs/compas_predictions.csv', index=False)

    pd.DataFrame({
        'prediction': adult_results['XGBoost']['predictions']
    }).to_csv('outputs/adult_predictions.csv', index=False)

    pd.DataFrame({
        'test_index': X_test_c.index
    }).to_csv('outputs/compas_test_indices.csv', index=False)

    pd.DataFrame({
        'test_index': X_test_a.index
    }).to_csv('outputs/adult_test_indices.csv', index=False)
    """)

    print("\n" + "="*70)
    print("DEMO: Running on Sample Data")
    print("="*70)

    # Create sample data for demonstration
    print("\nCreating sample data for demonstration...")

    np.random.seed(42)
    n_samples = 1000

    # Sample COMPAS-like data
    sample_compas = pd.DataFrame({
        'age': np.random.randint(18, 65, n_samples),
        'priors_count': np.random.poisson(2, n_samples),
        'race_binary': np.random.binomial(1, 0.7, n_samples),  # 70% privileged
        'sex_binary': np.random.binomial(1, 0.6, n_samples),
        'target': np.random.binomial(1, 0.45, n_samples)
    })

    # Create biased predictions (intentionally worse for unprivileged)
    sample_compas['prediction'] = sample_compas['target'].copy()
    # Add bias: increase false positives for unprivileged group
    bias_mask = (sample_compas['race_binary'] == 0) & (sample_compas['target'] == 0)
    sample_compas.loc[bias_mask, 'prediction'] = np.random.binomial(
        1, 0.3, bias_mask.sum()) # Use sum of bias_mask for size parameter

    print(f"✅ Created sample COMPAS data: {len(sample_compas)} records")

    # Analyze sample data
    sample_metrics = analyze_compas_fairness(
        sample_compas,
        sample_compas['prediction'].values,
        model_name="Sample_Demo"
    )

    print("\n" + "="*70)
    print("WEEK 2 SETUP COMPLETE")
    print("="*70)

    print("""
    ✅ AIF360 working correctly!
    ✅ Fairness metrics calculated successfully
    ✅ Visualizations generated

    📝 Next steps:
    1. Add save commands to Week 1 code (see instructions above)
    2. Re-run Week 1 to generate saved files
    3. Update this script to load your actual data
    4. Run full fairness analysis on all datasets

    Then move to Week 3: Bias Mitigation!
    """)


pip install 'aif360[inFairness]'


✅ AIF360 imported successfully!
✅ Fairlearn imported successfully!

FEDERAL AI BIAS RESEARCH - WEEK 2
Comprehensive Bias Detection & Fairness Metrics

⚠️  NOTE: This script requires Week 1 data.
Make sure you have:
  1. Trained models from Week 1
  2. Test predictions saved
  3. Preprocessed datasets

📂 Loading Week 1 results...

INSTRUCTIONS FOR RUNNING WEEK 2

    To run this analysis, you need to:
    
    1. Run Week 1 code first and save these variables:
       - Preprocessed dataframes
       - Model predictions
       - Test set indices
    
    2. Then load them here and run fairness analysis
    
    Example code to add at end of Week 1:
    
    # Save for Week 2
    compas_clean.to_csv('outputs/compas_clean.csv', index=False)
    adult_clean.to_csv('outputs/adult_clean.csv', index=False)
    
    pd.DataFrame({
        'prediction': compas_results['XGBoost']['predictions']
    }).to_csv('outputs/compas_predictions.csv', index=False)
    
    pd.DataFrame({
        'predictio

ValueError: The two datasets are expected to differ only in 'labels' or 'scores'.

FEDERAL AI BIAS DETECTION - MULTI-DOMAIN ANALYSIS
Analyzing 3 domains: Criminal Justice, Lending, Healthcare

DATASET 1: COMPAS - CRIMINAL JUSTICE RECIDIVISM

DATASET 2: ADULT INCOME - LENDING/CREDIT DECISIONS

DATASET 3: HEALTHCARE - DIABETES HOSPITAL READMISSION

STARTING MULTI-DOMAIN BIAS ANALYSIS

🔍 Analyzing Dataset 1/3: COMPAS (Criminal Justice)...

[1/6] Loading COMPAS dataset...
✓ Dataset loaded: 7214 records
[2/6] Preprocessing COMPAS data...
✓ Filtered: 6150 records
  - African-American: 3696
  - Caucasian: 2454
[3/6] Training COMPAS model...
✓ Model trained - Accuracy: 69.43%
[4/6] Calculating COMPAS bias metrics...

BIAS DETECTION RESULTS - COMPAS - Criminal Justice

📊 LEGAL COMPLIANCE CHECK:
   Disparate Impact: 1.862 (FAIL ✗)
   EEOC Threshold: 0.80 - 1.25 (legal range)

   ⚠️  LEGAL VIOLATION DETECTED
   Unprivileged group 1.86x more likely to get favorable outcome (reverse discrimination)

📈 DETAILED METRICS:
   Statistical Parity: 0.236
   Equal Opportunity: 0.165
   F